In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/My Drive/Federated Learning

[Errno 2] No such file or directory: '/content/gdrive/My Drive/Federated Learning'
/content


In [ ]:
%ls

ids-intrusion-csv.zip  kaggle.json


In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Federated Learning"

In [ ]:
!kaggle datasets download -d solarmainframe/ids-intrusion-csv

100% 1.60G/1.60G [00:22<00:00, 39.6MB/s]
100% 1.60G/1.60G [00:22<00:00, 75.3MB/s]


In [ ]:
!unzip *.zip

Archive:  ids-intrusion-csv.zip
  inflating: 02-14-2018.csv          
  inflating: 02-15-2018.csv          
  inflating: 02-16-2018.csv          
  inflating: 02-20-2018.csv          
  inflating: 02-21-2018.csv          
  inflating: 02-22-2018.csv          
  inflating: 02-23-2018.csv          
  inflating: 02-28-2018.csv          
  inflating: 03-01-2018.csv          
  inflating: 03-02-2018.csv          


In [ ]:
pip uninstall scipy

Found existing installation: scipy 1.7.3
Uninstalling scipy-1.7.3:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/scipy-1.7.3.dist-info/*
    /usr/local/lib/python3.10/dist-packages/scipy.libs/libgfortran-040039e1.so.5.0.0
    /usr/local/lib/python3.10/dist-packages/scipy.libs/libopenblasp-r0-8b9e111f.3.17.so
    /usr/local/lib/python3.10/dist-packages/scipy.libs/libquadmath-96973f99.so.0.0.0
    /usr/local/lib/python3.10/dist-packages/scipy/*
Proceed (Y/n)? y
  Successfully uninstalled scipy-1.7.3


In [ ]:
pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scipy-1.10.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.4 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dp-accounting 0.4.1 requires scipy~=1.7.1, but you have scipy 1.10.1 which is incompatible.
orbax-checkpoint 0.2.1 requires jax>=0.4.8, but you have jax 0.3.15 which is incompatible.
pymc 5.1.2 requires cachetools>=4.2.1, but you have cachetools 3.1.1 which is incompatible.


In [ ]:
pip install tensorflow-federated --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scipy-1.7.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (39.9 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
arviz 0.15.1 requires scipy>=1.8.0, but you have scipy 1.7.3 which is incompatible.
orbax-checkpoint 0.2.1 requires jax>=0.4.8, but you have jax 0.3.15 which is incompatible.
pymc 5.1.2 requires cachetools>=4.2.1, but you have cachetools 3.1.1 which is incompatible.


In [4]:
# Data processing
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.preprocessing import MinMaxScaler

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Model and performance
import tensorflow as tf
from tensorflow.python.keras import layers, losses
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics as metrics

RANDOM_STATE_SEED = 12

In [5]:
df = pd.read_csv('/content/gdrive/My Drive/Kaggle/02-14-2018.csv')
df = df.drop(['Timestamp'], axis=1)
df.replace([-np.inf, np.inf], np.nan, inplace = True)
df.dropna(inplace=True)
df.replace(to_replace=["FTP-BruteForce", "SSH-Bruteforce"], value="Malicious", inplace=True)
df.replace(to_replace = 'Benign', value = 0, inplace=True)
df.replace(to_replace = 'Malicious', value = 1, inplace=True)

In [6]:
scaler = MinMaxScaler()
scaler.fit(df)
df = pd.DataFrame(scaler.transform(df), columns=df.columns)

In [7]:
from google.colab import files
df.to_csv('dataset1.csv', encoding = 'utf-8-sig')
files.download('dataset1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Initialize the scaler
scaler = MinMaxScaler()
scaler.fit(df)
df = pd.DataFrame(scaler.transform(df), columns=df.columns)
dfmal = df.loc[df['Label'] == 1]
dfben = df.loc[df['Label'] == 0]

In [ ]:
dfben.drop('Label', axis=1, inplace=True)
dfmal.drop('Label', axis=1, inplace=True)
dfben = dfben.values
dfmal = dfmal.values

In [ ]:
train, test = train_test_split(dfben, test_size=0.25, random_state=42)

In [ ]:
model = tf.keras.models.load_model('saved_model/d1_trained_model.h5')

In [ ]:
'''
import os

import tensorflow as tf
from tensorflow import keras
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)
'''

In [ ]:
'''
from sklearn import metrics
import numpy as np
import pandas as pd
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(78, input_dim=train.shape[1], activation='sigmoid'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='sigmoid'))
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(16, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(train.shape[1]))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()
'''

In [ ]:
'''
model.fit(train, train, verbose=1, epochs=10, callbacks=[cp_callback])
'''

In [ ]:
pred = model.predict(train)
score1 = np.sqrt(metrics.mean_squared_error(pred, train))

In [ ]:
pred = model.predict(test)
score2 = np.sqrt(metrics.mean_squared_error(pred, test))

In [ ]:
pred = model.predict(dfmal)
score3 = np.sqrt(metrics.mean_squared_error(pred, dfmal))

In [ ]:
pred = model.predict(dfben)
score4 = np.sqrt(metrics.mean_squared_error(pred, dfben))

In [ ]:
print(f"Train Dataset Score (RMSE): {score1}".format(score1))
print(f"Test Dataset Score (RMSE): {score2}")
print(f"Malicious Dataset Score (RMSE): {score3}")
print(f"Benign Dataset Score (RMSE): {score4}")

In [ ]:
copydf = df
copydf.drop('Label', axis=1, inplace=True)
copydf = copydf.values

pred = model.predict(copydf)

scores = []
predictions = []
for i in range(len(copydf)):
  scores.append(np.sqrt(metrics.mean_squared_error(pred[i], copydf[i])))
  if scores[i] < 0.05:
    predictions.append('Benign')
  else:
    predictions.append('Malicious')
  if ((i/len(copydf)) * 100).is_integer():
    print(((i/len(copydf)) * 100), '% complete')

In [ ]:
df = pd.read_csv('/content/gdrive/My Drive/Kaggle/02-14-2018.csv')
df = df.drop(['Timestamp'], axis=1)
df.replace([-np.inf, np.inf], np.nan, inplace = True)
df.dropna(inplace=True)
df.replace(to_replace=["FTP-BruteForce", "SSH-Bruteforce"], value="Malicious", inplace=True)
df.replace(to_replace = 'Benign', value = 0, inplace=True)
df.replace(to_replace = 'Malicious', value = 1, inplace=True)
scaler.fit(df)
df = pd.DataFrame(scaler.transform(df), columns=df.columns)

In [ ]:
tp = 0
fn = 0
fp = 0
tn = 0

for i in range(len(predictions)):
  if predictions[i] == 'Malicious' and df['Label'][i] == 1:
    tp += 1
  elif predictions[i] == 'Malicious' and df['Label'][i] == 0:
    fp += 1
  elif predictions[i] == 'Benign' and df['Label'][i] == 0:
    tn += 1
  else:
    fn += 1

In [ ]:
precision = tp / (fp + tp)
recall = tp / (fn + tp)
accuracy = ( tp + tn ) / (tp + tn + fp + fn)
f1score = (2 * precision * recall) / (precision + recall)

print("Precision : ", precision)
print("Recall    : ", recall)
print("Accuracy  : ", accuracy)
print("F1 Score  : ", f1score)

In [ ]:
cm_data = [[tn, fp], [fn, tp]]
sns.heatmap(cm_data, annot=True, cmap='Blues', fmt='d')

In [ ]:
pip install tensorflow-federated

In [ ]:
import tensorflow as tf
import tensorflow_federated as tff


# Define a new uncompiled Keras model with the same architecture and weights as your existing model
uncompiled_model = tf.keras.Sequential.from_config(model.get_config())
uncompiled_model.set_weights(model.get_weights())

# Define the loss function and input spec
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
input_spec = (tf.TensorSpec(shape=(None, None, 78), dtype=tf.float32), tf.TensorSpec(shape=(None,), dtype=tf.int32))

# Wrap your uncompiled Keras model with the tff.learning.from_keras_model function to create a TFF model
def model_fn():
    keras_model = uncompiled_model
    return tff.learning.from_keras_model(
        keras_model=keras_model,
        input_spec=input_spec,
        loss=loss_fn)

tff_model = model_fn()

# Define your data
client_data = ...

# Set up your Federated Learning (FL) experiment
num_rounds = ...
eval_fn = ...
client_sampler = ...

# Train your model
for round_num in range(num_rounds):
    # Select clients for this round
    client_ids = client_sampler.sample()

    # Create a new tff.learning.ClientData object for the selected clients
    federated_train_data = [client_data.create_tf_dataset_for_client(client_id) for client_id in client_ids]
    client_batch = [preprocess(federated_train_data[i].take(5)) for i in range(len(federated_train_data))]

    # Create a new tff.templates.IterativeProcess object for each round
    iterative_process = tff.learning.build_federated_averaging_process(
        tff_model, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.1))

    # Run the training process
    state, metrics = iterative_process.next(state, client_batch)

    # Evaluate the model on the test data
    eval_metrics = eval_fn(state.model)

# Evaluate your model
test_metrics = eval_fn(state.model)
